In [ ]:
nb_of_edf_homologs_file = 'edf_occurence_table.tsv'
nb_of_edf_homologs = {}
with open(nb_of_edf_homologs_file) as h:
    next(h)
    for l in h:
        l = l.strip().split('\t')
        assert l[0] not in nb_of_edf_homologs
        nb_of_edf_homologs[l[0]] = int(l[1])


scaffold2anc_nb = Counter()  # numbers of proteins in at least 2 fungi per scaffold
for acc in nb_of_edf_homologs:
    if nb_of_edf_homologs[acc] >= 2:
        try:
            scaffold = acc2scaffold[acc]
        except KeyError:
            continue
        scaffold2anc_nb[scaffold] += 1

In [ ]:
acc2scaffold_file = '../Raw_data/acc2scaffold.tsv'
acc2scaffold = {}
with open(acc2scaffold_file) as h:
    for l in h:
        l = l.strip().split('\t')
        assert l[0] not in acc2scaffold
        acc2scaffold[l[0]] = l[1]
len(acc2scaffold)  # note: with piromyces sp E2

In [ ]:
# Remove potential contaminants from clusters
contaminants = 0
for k in cluster_table:
    seqids = cluster_table[k]
    to_retain = []
    for sid in seqids:
        if sid in target_accessions:
            on_ancestral_contig = scaffold2anc_nb[acc2scaffold[sid]] >= 1
        else:  # we don't care about non-target proteins
            on_ancestral_contig = True
        to_retain.append(on_ancestral_contig)
        contaminants += 1-on_ancestral_contig
    cluster_table[k] = [sid for sid, anc in zip(seqids, to_retain) if anc]
print('Removed %i possibly contaminating sequences' % contaminants)